In [1]:
import pandas as pd
import math
from geopy.distance import great_circle

In [2]:
mapa = pd.read_csv("mapa_mexico_w_lz_zonas_excluidas.csv")
solar = pd.read_csv("solar_generation_w_lz.csv")
subestaciones = pd.read_csv("subestaciones_w_lz.csv")

In [12]:
mapa

,LAT,LON,lz,GENERATION,NEAREST_SUBSTATION
0,19.783276,-88.170226,43.0,2.829711e+06,97.605046
1,17.025762,-91.234932,38.0,2.796227e+06,102.599975
2,29.988325,-106.002080,8.0,3.361254e+06,70.802313
3,28.050810,-100.566989,12.0,3.128374e+06,84.903619
4,18.425554,-99.616072,31.0,3.162039e+06,39.830710
5,21.277399,-98.140022,20.0,3.003276e+06,79.737991
6,22.435550,-101.863349,24.0,3.221722e+06,53.033660
7,28.004805,-103.299523,12.0,3.214423e+06,144.922646
8,25.709937,-98.988149,14.0,3.046445e+06,31.135090
9,22.151659,-101.941799,24.0,3.228868e+06,41.510419


In [3]:
add_to_mapa = []
for i, row in mapa.iterrows():
    add_to_mapa.append(solar.iloc[int(row[2])][3])
mapa['GENERATION'] = add_to_mapa

In [4]:
del mapa['X']
del mapa['Y']
del mapa['field_1']

In [ ]:
distance_to_nearest_lz_substation_km = []
subestaciones = subestaciones.sort_values(['lz'], ascending = [True])

for i, row in mapa.iterrows():
    min_distance = 1000000000.0
    for _i, _row in subestaciones.iterrows():
        if _row['lz'] > row['lz']:
            break
        if _row['lz'] == row['lz']:
            if great_circle((row['LAT'], row['LON']), (_row['Y'], _row['X'])).kilometers < min_distance:
                min_distance = great_circle((row['LAT'], row['LON']), (_row['Y'], _row['X'])).kilometers
    distance_to_nearest_lz_substation_km.append(min_distance)

In [ ]:
mapa['NEAREST_SUBSTATION'] = distance_to_nearest_lz_substation_km

In [ ]:
ratio = []
for i, row in mapa.iterrows():
    ratio.append(row['GENERATION']/row['NEAREST_SUBSTATION'])
mapa['RATIO'] = ratio

In [15]:
mapa.to_csv("mexico_w_lz_ratio.csv")

In [1]:
mapa

NameError: name 'mapa' is not defined